In [ ]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder

# Step 1: Load transactions from CSV
def load_transactions(file_path):
    df = pd.read_csv(file_path, header=None)
    transactions = df.apply(lambda row: row.dropna().tolist(), axis=1).tolist()
    return transactions

# Step 2: Apply Apriori algorithm and generate rules
def run_apriori(transactions, min_support=0.004, min_confidence=0.2, min_lift=3, min_len=2):
    te = TransactionEncoder()
    te_ary = te.fit(transactions).transform(transactions)
    df = pd.DataFrame(te_ary, columns=te.columns_)

    # Frequent itemsets
    frequent_itemsets = apriori(df, min_support=min_support, use_colnames=True)

    # Association rules
    rules = association_rules(frequent_itemsets, metric="lift", min_threshold=min_lift)

    # Filter by confidence and length
    rules = rules[
        (rules['confidence'] >= min_confidence) &
        (rules['lift'] >= min_lift) &
        (rules['antecedents'].apply(lambda x: len(x) >= 1)) &
        (rules['consequents'].apply(lambda x: len(x) >= 1)) &
        ((rules['antecedents'].apply(len) + rules['consequents'].apply(len)) >= min_len)
    ]

    return rules

# Step 3: Display rules
def display_rules(rules):
    if rules.empty:
        print("No association rules found with the given thresholds.")
        return
    print(f"{'Rule':50s}  {'Support':>8s}  {'Confidence':>10s}  {'Lift':>8s}")
    print("-" * 85)
    for _, row in rules.iterrows():
        rule = f"{set(row['antecedents'])} => {set(row['consequents'])}"
        print(f"{rule:50s}  {row['support']:.4f}    {row['confidence']:.4f}    {row['lift']:.4f}")

# Main Execution
if __name__ == "__main__":
    file_path = 'transactions.csv'  # <-- Ensure your CSV file is in the same folder
    transactions = load_transactions(file_path)
    rules = run_apriori(transactions,
                        min_support=0.0040,
                        min_confidence=0.2,
                        min_lift=3,
                        min_len=2)
    display_rules(rules)
